<a href="https://colab.research.google.com/github/sko9370/rootCauseAnalysis/blob/main/Root_Cause_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download and uncompress the accumulated log
!wget https://github.com/sko9370/rootCauseAnalysis/raw/main/host.zip
!unzip host.zip
!rm host.zip

In [ ]:
# install package to be able to load in json logs
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

# ingest host.json log into a spark dataframe
sparkdf = spark.read.json("host.json")
# create table compatible with SQL queries
sparkdf.createTempView("host")

In [ ]:
# run a sql query on previously ingested log. use the name from the .createTempView method
# DESCRIBE is a sql function that lists all the columns and the type of data they use
describe = spark.sql(
'''
DESCRIBE host
''')
# .show(<number of rows to show>, <truncate> means to cut off columns to make them fit on the screen)
# setting vertical to true may be easier to view (similar to default Splunk view) if there are a lot of columns
describe.show(40, truncate = 70, vertical = False)

In [ ]:
# a little bit more complex query that aggregates Image names in sysmon log entries and outputs in ascending order
process = spark.sql(
'''
SELECT Image, count(*) as count
FROM host
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
GROUP BY Image
ORDER BY count ASC
''')
process.show(40, truncate = 70)

In [ ]:
# another method to query using a more code-like syntax if you are unfamiliar with sql. very similar keywords however
# also note that this method uses the dataframe directly instead of the temporary view (sql)
registryValues = sparkdf.filter("Channel == 'Microsoft-Windows-Sysmon/Operational' AND EventID == 13")\
    .groupBy('TargetObject').count().orderBy('count',ascending = False).show(truncate = 90)

In [ ]:
# types of log entires available
channels = spark.sql(
'''
SELECT Channel, count(Channel) as count
FROM host
GROUP BY Channel
ORDER BY count DESC
'''
)
channels.show(100, truncate = False)

In [ ]:
# alternate method for same thing
sources = sparkdf.groupBy('Channel').count().orderBy('count',ascending = False).show(truncate = False)